We seek to reweight a portfolio of weights to be "beta-neutral" against another ETF.

In practice, we will be making our Silver Fund portfolio beta-neutral against AI ETFs

In [62]:
import numpy as np

In [63]:
import requests
import datetime as dt
import polars as pl

In [64]:
from collections import defaultdict

In [65]:
import sf_quant as sf
import sf_quant.data as sfd


In [66]:
class Portfolio:
    def __init__(self):
        """
        Initial Silver Fund weights to be re-weighted according to our views on other ETFs.
        """
        self.date_ = dt.date(2025, 11, 18)
        self.date = str(self.date_)
        self.holdings = self._get_holdings_and_weights()

        columns = ["barrid", "ticker"]
        self.date2 = dt.date(2025, 1, 13)
        df = sf.data.load_assets_by_date(
            date_=self.date2,
            in_universe=True,
            columns=columns
        )
        self.ticker_to_barrid = dict(zip(df["ticker"], df["barrid"]))
        self.barrid_to_ticker = dict(zip(df["barrid"], df["ticker"]))
        
        self._get_blackrock_etf()
        
    def _calculate_beta(self):
        """
        Calculate beta of our holdings against our ETF
        """
        potential_tickers = set(self.holdings.keys()) | set(self.etf.keys())
        all_tickers = sorted([t for t in potential_tickers if t in self.ticker_to_barrid])
        barrids = [self.ticker_to_barrid[ticker] for ticker in all_tickers]
        V = sfd.construct_covariance_matrix(self.date2, barrids).to_numpy()[:,1:] / 252
        w_port = np.array([self.holdings.get(t, 0.0) for t in all_tickers])
        w_etf = np.array([self.etf.get(t, 0.0) for t in all_tickers])
        covariance_port_etf = w_port.T @ V @ w_etf
        variance_etf = w_etf.T @ V @ w_etf
        if variance_etf == 0:
            return 0.0
        beta = covariance_port_etf / variance_etf
        self.beta = beta
        print(f"Portfolio Beta against ETF: {beta:.4f}")
        return beta

    def rebalance(self):
        """
        Rebalance our weights against another ETF.
        """

        target_etf_weights = self.etf

        # all potential tickers
        potential_tickers = set(self.holdings.keys()) | set(target_etf_weights.keys())
        
        # filter out tickers not in out ticker to barrid map
        all_tickers = sorted([t for t in potential_tickers if t in self.ticker_to_barrid])
        dropped_tickers = potential_tickers - set(all_tickers)
        if dropped_tickers:
            print(f"Dropped {len(dropped_tickers)} tickers due to missing data: {list(dropped_tickers)}")

        barrids = [self.ticker_to_barrid[ticker] for ticker in all_tickers]
        V = sfd.construct_covariance_matrix(self.date2, barrids).to_numpy()[:,1:] / 252
        w_f = np.array([target_etf_weights.get(t, 0.0) for t in all_tickers])
        numerator = V @ w_f
        denominator = w_f.T @ V @ w_f
        rebalance_vector = numerator / denominator
        self.new_holdings = defaultdict(float, dict(zip(all_tickers, rebalance_vector)))
        return self.new_holdings
    
    def _get_holdings_and_weights(self):
        """
        Helper to fetch dictionary of current holdings.
        """
        date_ = self.date
        url = "https://prod-api.silverfund.byu.edu"
        endpoint = "/all-holdings/summary"
        json = {"start": date_, "end": date_, "fund": "quant_paper"}
        response = requests.post(url + endpoint, json=json)
        if not response.ok:
            raise Exception(response.text)
        holdings = pl.DataFrame(response.json()["holdings"])
        holdings = holdings.with_columns([
            (pl.col("shares") * pl.col("price")).alias("market_value")
        ])
        total_value = holdings["market_value"].sum()
        holdings = holdings.with_columns([
            (pl.col("market_value") / total_value).alias("weight")
        ])
        return defaultdict(float, zip(holdings["ticker"], holdings["weight"]))

    def _get_janus_etf(self):
        ai_etf = pl.read_csv(
            'janus_henderson_ai_etf.csv',
            skip_rows=3,
            has_header=False,
        )

        column_names = [
            "underlying_security", "ticker", "cusip", "underlying_security_type",
            "strike_price", "quantity", "notional_value", "market_value", 
            "weight_percent", "current_market_value"
        ]

        ai_etf = ai_etf.select(pl.all().slice(0, 46))
        ai_etf = ai_etf.rename(dict(zip(ai_etf.columns, column_names)))

        ai_etf = ai_etf.with_columns([
            pl.col("quantity").str.replace_all(",", "").cast(pl.Int64, strict=False),
            pl.col("notional_value").str.replace_all("[$|,]", "").cast(pl.Float64, strict=False),
            pl.col("market_value").str.replace_all("[$|,]", "").cast(pl.Float64, strict=False),
            (pl.col("weight_percent").cast(pl.Float64, strict=False) / 100).alias("weight_decimal"),
            pl.col("current_market_value").str.replace_all("[$|,]", "").cast(pl.Float64, strict=False)
        ]).drop("weight_percent")

        weights_dict = defaultdict(lambda: 0.0)

        for row in ai_etf.rows(named=True):
            ticker = row.get("ticker")
            ticker = ticker.removesuffix(" US") if ticker.endswith(" US") else ticker
            weight = row.get("weight_decimal", 0.0)
            if ticker:
                weights_dict[ticker] = weight

        self.etf = weights_dict
        return self.etf

    def _get_blackrock_etf(self):
        etf = pl.read_csv('BAI_holdings.csv')

        # Clean the data - filter out rows where weight is not numeric
        etf_clean = etf.filter(
            pl.col("Weight (%)").is_not_null() & 
            pl.col("Weight (%)").is_not_nan()
        )

        # Convert weight from percent to float
        etf_clean = etf_clean.with_columns(
            (pl.col("Weight (%)") / 100).alias("weight")
        )

        weights_dict = defaultdict(lambda: 0.0)

        for row in etf_clean.rows(named=True):
            ticker = row.get("Ticker")
            weight = row.get("weight", 0.0)
            if ticker:
                weights_dict[ticker] = weight

        self.etf = weights_dict

    @staticmethod
    def _get_weights_array(self):
        """
        Helper to get weights as a numpy array for matrix operations.
        Ensures consistent ordering (sorted by ticker) for matrix alignment.
        """
        sorted_weights = [self.holdings[ticker] for ticker in sorted(self.holdings.keys())]
        return np.array(sorted_weights)


In [ ]:
portfolio = Portfolio()

In [ ]:
portfolio.etf

defaultdict(<function __main__.Portfolio._get_blackrock_etf.<locals>.<lambda>()>,
            {'AVGO': 0.0992,
             'NVDA': 0.091,
             'MSFT': 0.0546,
             'META': 0.0444,
             'SNOW': 0.0443,
             'CLS': 0.037700000000000004,
             'GOOGL': 0.0371,
             'TSM': 0.0327,
             'FN': 0.031200000000000002,
             'ORCL': 0.028399999999999998,
             'TSEM': 0.027999999999999997,
             'CRDO': 0.0279,
             '9984': 0.0278,
             '6857': 0.0278,
             'AMD': 0.0269,
             'NET': 0.0234,
             'LRCX': 0.0233,
             'APP': 0.023,
             'PSTG': 0.0222,
             'TSLA': 0.0211,
             'ANET': 0.0176,
             'MPWR': 0.0174,
             'PLTR': 0.0172,
             'ENR': 0.0168,
             'BWXT': 0.016,
             '3661': 0.0149,
             '007660': 0.013600000000000001,
             'KTOS': 0.0126,
             '034020': 0.0121,
             

In [ ]:
portfolio._calculate_beta()

Portfolio Weight Sum: 0.8915
Portfolio Beta against ETF: 0.6248


np.float64(0.6248401858895252)

In [ ]:
portfolio.holdings

defaultdict(float,
            {'MSFT': 0.06488426463583538,
             'NVDA': 0.0571939175812638,
             'AAPL': 0.05408774631276599,
             'LRN': 0.038955067835718754,
             'AMZN': 0.03534611245872022,
             'BRK B': 0.03457799070125,
             'META': 0.031414711622505476,
             'GOOG': 0.028001520173669633,
             'GOOGL': 0.02728505596124252,
             'MA': 0.022304336246674817,
             'FMC': 0.021296654195160403,
             'VRSN': 0.019110446558956502,
             'TR': 0.018258959738749168,
             'NFLX': 0.01825050440069743,
             'CD': 0.017888181739048034,
             'FFIV': 0.01738183267720656,
             'TXN': 0.016178032487421883,
             'OGN': 0.015892424879393238,
             'BX': 0.015756099692507748,
             'ADI': 0.015251076062036547,
             'FISV': 0.01506889780541914,
             'CSGP': 0.014351542354656846,
             'DRS': 0.013053213770680255,
             'SFM

In [ ]:
portfolio._calculate_beta()

Portfolio Weight Sum: 0.8915
Portfolio Beta against ETF: 0.6248


np.float64(0.6248401858895252)

In [ ]:
portfolio.holdings["NVDA"]

0.0571939175812638

In [ ]:
portfolio.rebalance()

Dropped 41 tickers due to missing data: ['XTSLA', 'FISV', 'SFD', '3661', '9984', '034020', '000660', 'SOLS', 'SNWV', 'MONDQ', 'AARD', 'AUD', 'NIQ', 'BKKT', 'TR', 'BRK B', 'EUR', 'KRW', 'CMDB', 'FLEX', '007660', 'TNXP', 'TWD', 'SHOP', 'USD', 'RCMT', 'BGXXQ', 'ETON', 'CD', 'TSM', 'TSEM', 'PSTX.CVR', 'RBRK', 'VELO', 'NAGE', '6857', 'TEM', 'CHWY', 'PME', 'JPY', 'CLS']


defaultdict(float,
            {'AAPL': 0.8575956345942426,
             'ABBV': 1.0113862250289454,
             'ABT': 1.0075832756284555,
             'ADI': 2.1977610319084886,
             'ALAB': 1.8281412395586982,
             'ALNY': 1.0105537339372377,
             'AM': 1.8746017158640003,
             'AMD': 1.554649288192777,
             'AMZN': 0.9693690705572503,
             'ANET': 1.0402168860204548,
             'APP': 1.9653759192226776,
             'APPF': 0.9099840249359813,
             'ARCT': 0.6957110341926496,
             'AVGO': 1.3886985543264811,
             'BA': 1.0356126275479922,
             'BAH': 1.3710947219298297,
             'BLBD': 1.9214845266253684,
             'BLK': 1.0501585151664834,
             'BWXT': 1.2553664297718385,
             'BX': 0.9992703331794812,
             'CASY': 1.1623975501314603,
             'CIEN': 0.745421934268247,
             'CMCL': 0.42111475454356523,
             'COP': 0.3427922911199626,
           

In [ ]:
portfolio.new_holdings["NVDA"]

1.692901969826908

In [ ]:
def _get_holdings():
    date_ = str(dt.date(2025, 11, 18))

    url = "https://prod-api.silverfund.byu.edu"
    endpoint = "/all-holdings/summary"
    json = {"start": date_, "end": date_, "fund": "quant_paper"}

    response = requests.post(url + endpoint, json=json)

    if not response.ok:
        raise Exception(response.text)

    holdings = pl.DataFrame(response.json()["holdings"])

    # Calculate Market Values and Weights
    holdings = holdings.with_columns([
        (pl.col("shares") * pl.col("price")).alias("market_value")
    ])
    
    total_value = holdings["market_value"].sum()
    
    holdings = holdings.with_columns([
        (pl.col("market_value") / total_value).alias("weight")
    ])

    return holdings

holdings = _get_holdings()
holdings

ticker,active,shares,price,value,total_return,volatility,dividends,dividends_per_share,dividend_yield,alpha,beta,market_value,weight
str,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""MSFT""",true,115.0,493.79,56785.85,-2.699561,0.0,0.0,0.0,0.0,-2.711655,0.000197,56785.85,0.064884
"""NVDA""",true,276.0,181.36,50055.36,-2.808146,0.0,0.0,0.0,0.0,-2.820234,0.000205,50055.36,0.057194
"""AAPL""",true,177.0,267.44,47336.88,-0.007478,0.0,0.0,0.0,0.0,-0.019714,0.000001,47336.88,0.054088
"""LRN""",true,528.0,64.57,34092.96,2.916799,0.0,0.0,0.0,0.0,2.904409,-0.000211,34092.96,0.038955
"""AMZN""",true,139.0,222.55,30934.45,-4.431657,0.0,0.0,0.0,0.0,-4.44366,0.000323,30934.45,0.035346
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""RJF""",true,1.0,155.78,155.78,0.28325,0.0,0.0,0.0,0.0,0.270998,-0.00002,155.78,0.000178
"""PSTX.CVR""",true,1.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,-0.012237,8.8897e-7,0.5,5.7131e-7
"""VELO""",true,0.0667,4.41,0.294147,1.37931,0.0,0.0,0.0,0.0,1.367001,-0.000099,0.294147,3.3610e-7
